# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [28]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [29]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [30]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
parquet_files

['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [31]:
# Write your code below.

# Reading parquet files to a dask object
dd_px = dd.read_parquet(parquet_files)

#Addind lag for variable Close 
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close_lag_1 = x['Adj Close'].shift(1))
).assign(
    #Adding returns based on Close
    returns = lambda x: (x['Close']/x['Close_lag_1']) - 1
).assign(
    #Adding the hi_lo_range
    hi_lo_range = lambda x: x['High'] - x['Low'] 
))

dd_feat


C:\Users\rabia\AppData\Local\Temp\ipykernel_11824\3249180066.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=3169,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [32]:
# Write your code below.

# Converting dask data frame to pandas data frame
df = dd_feat.compute()

# Sorting data frame by 'ticker' and 'date'
df = df.sort_values(['ticker', 'Date'], ascending = [True, True], ignore_index=True)

#Adding a moving average of returns calculation using a window of 10 days
df['Moving_Avg_10'] = df.groupby('ticker', sort =False)['returns'].rolling(window=10).mean().reset_index(level=0, drop=True)

df

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,Moving_Avg_10
0,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,ACN,2001,188.600006,187.069931,-0.919565,0.29,NaN
1,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,ACN,2001,15.170000,11.404394,-0.010547,0.25,NaN
2,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,ACN,2001,15.010000,11.284108,-0.000666,0.46,NaN
3,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,ACN,2001,15.000000,11.276587,-0.009333,0.27,NaN
4,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,ACN,2001,14.860000,11.171341,0.006057,0.30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371605,2020-03-26,4.06,4.53,3.88,4.51,4.510000,1668500.0,ZIXI.csv,ZIXI,2020,4.000000,4.000000,0.127500,0.65,-0.000807
371606,2020-03-27,4.49,4.71,4.10,4.60,4.600000,1146800.0,ZIXI.csv,ZIXI,2020,4.510000,4.510000,0.019956,0.61,0.004262
371607,2020-03-30,4.83,4.87,4.44,4.64,4.640000,1212000.0,ZIXI.csv,ZIXI,2020,4.600000,4.600000,0.008696,0.43,0.015491
371608,2020-03-31,4.60,4.69,4.10,4.31,4.310000,1057200.0,ZIXI.csv,ZIXI,2020,4.640000,4.640000,-0.071121,0.59,0.015926


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Answer: No, it was not necessary. Dask also supports rolling operations, so the moving average could be calculated directly on the Dask DataFrame without converting to pandas.

+ Would it have been better to do it in Dask? Why?

Answer: It depends on the use case. If memory efficiency or scalability is important, especially with large datasets, it's better to perform the rolling calculation using Dask. However, since this dataset is relatively small and fits comfortably in memory, converting to pandas for simplicity and faster execution makes sense in this context.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.